# Extreme $Re$ Newtonian Similarity

As usual, we will begin by importing the necessary python packages

In [ ]:
from barennet import SimilarityModel
import tensorflow as tf
import numpy as np

# Uncomment the line below if if you have tensorflow issues regarding your GPU:

# tf.config.set_visible_devices([], 'GPU')

We will find the incomplete similarity exponents for the extreme $Re$ newtonian 
mean velocity profile in wall coordinates. We know that the dimensionless quantities
involved are $(u^+, y^+, Re_\tau)$ and, although we don't know the relation between
them, we have reason to believe that they have the following similarity relation:

$$ u^+ = Re_\tau^{1/12} \Phi^{(1)} \left( \frac{y^+}{Re_\tau} \right)$$

We provide a data file with experimental MVP data by McKeon et. al. collected at
the Princeton Superpipe facility. We will use Reynolds numbers in the range 
$2.3 \times 10^6 < Re < 1.3 \times 10^7$. The MDDP construction of the three 
dimensionless quantities is also provided in the code cell below.

In [ ]:
data_path = "../Data/extreme_re_MVP_data.xlsx"
dimensionally_independent_parameters = ["mu", "rho", "delta"]
dimensionally_dependent_parameters = ["y", "- dp/dz"]
dimensional_qoi = "u"
non_dimensional_parameters = ["y+", "Re_tau"]
non_dimensional_qoi = "u+"
non_dimensional_params_construction = { 
    "y+": {"y": 1.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 0.5},
    "Re_tau": {"y": 0.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 1.5}, 
}
non_dimensional_qoi_construction = {
    "u+": {"u": 1, "y": 0.0, "- dp/dz": -0.5, "mu": 0.0, "rho": 0.5, "delta": -0.5}
}
non_similar_params = ["y+"]
similar_params = ["Re_tau"]

In [ ]:
similarity_model = SimilarityModel(
    data_path=data_path,
    dimensionally_independent_params=dimensionally_independent_parameters,
    dimensionally_dependent_params=dimensionally_dependent_parameters,
    dimensional_qoi=dimensional_qoi,
    non_dimensional_params=non_dimensional_parameters,
    non_dimensional_qoi=non_dimensional_qoi,
    non_dimensional_params_construction=non_dimensional_params_construction,
    non_dimesional_qoi_construction=non_dimensional_qoi_construction,
    non_similar_params=non_similar_params,
    similar_params=similar_params)

### Buckingham's Similarity Group

As soon as the dimensionless construction is provided and we create an instance 
of the SimilarityModel class, the Buckingham's similarity group can be calculated
by solving a linear system (chapter 5 of the thesis).
It shoud read:
\begin{align*}
    &\mu^* = A_1 \mu, \ \ \ \rho^* = A_2 \rho, \ \ \ \delta^* = A_3 \delta \\ & \\
    & y^* = A_3 y \\ & \\
    & \left( \frac{dp}{dz} \right)^* = \frac{A_1^2}{A_2 A_3^3} \left( \frac{dp}{dz} \right) \\ & \\
    &U^* = \frac{A_1}{A_2 A_3} U
\end{align*}

And, as you can see, this is exactly the output of the cell below.

In [ ]:
similarity_model.print_buckingham_similarity_group()

We can now use the find_incomplete_similarity method in order to use the BarenNet to find proper incomplete similarity exponents, as well as estimating the function $\Phi^{(1)}$ with a Deep Neural Network.

In [ ]:
similarity_model.find_incomplete_similarity(n_tries=5, n_epochs=10000, tol=5e-3)

After training has been performed, we can check the exponents found by the BarenNet by printing the dictionary used to store them in our class. 

$\cdot$ The entry dict["y+"]["Re_tau"] refers to the exponent of $Re_\tau$ when it is multiplied by $y^+$ i.e. $\xi_2^{(1)}$;

$\cdot$ As one should expect, the entry dict["u+"]["Re_tau"] refers to the exponent of $Re_\tau$ when it is multiplied by $u^+$ i.e. $\xi_1$.

By looking at the incomplete similarity theoretical equation above, we come to the conclusion that $\xi_1 \approx - 1/12$ and  $\xi_2^{(1)} \approx -1$. Let's see:

In [ ]:
print(similarity_model.incomplete_similarity_exponents_dict)

Now that incomplete similarity has been found, our package has already automatically calculated the renormalization group from the exponents found. Much like Buckingham's similarity group, we can print it with the following method:

In [ ]:
similarity_model.print_renormalization_group()

Now we will do plot some data points in original coordinates and, afterwards, plot the same points in the renormalized dimensionless coordinates. A data collapse is expected.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
plt.rc('mathtext', fontset="cm")
import matplotlib.ticker as mtick
from matplotlib import cm # Colormaps
import matplotlib.gridspec as gridspec
import seaborn as sns

from tutorial_utils.plotting_methods import (plot_extreme_re_flow,
                                             plot_extreme_re_flow_renormalized)

In [ ]:
plot_extreme_re_flow()

In [ ]:
plot_extreme_re_flow_renormalized(similarity_model.incomplete_similarity_exponents_dict)